In [1363]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures


In [1364]:
RANDOM_STATE = 42

In [1365]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target
X

/Users/17299765/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets inclu

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [1366]:
learn_df = X.sample(frac = 0.8)
test_df = X.drop(learn_df.index)
print('Size of learn df: '+ str(len(learn_df))+ ', size of test df: '+str(len(test_df)))
learn_y, test_y = np.split(y,[int(len(y)*0.8)+1])       # получаем идентичную разбивку df & y
print('Size of learn_y: '+str(len(learn_y)) + ', size of test y: '+str(len(test_y)))

Size of learn df: 405, size of test df: 101
Size of learn_y: 405, size of test y: 101


2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [1367]:
reg = LinearRegression().fit(learn_df, learn_y)
predicted_y = reg.predict(test_df)
r2_score(test_y, predicted_y)

-2.2690186786692306

In [1368]:
ridge = Ridge()
ridge.fit(learn_df, learn_y)
predicted_ridge_y = ridge.predict(test_df)
r2_score(test_y, predicted_ridge_y)

-2.2838629255786675

In [1369]:
lasso = Lasso()
lasso.fit(learn_df, learn_y)
predicted_lasso_y = lasso.predict(test_df)
r2_score(test_y,predicted_lasso_y)

-2.3207097413954756

3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [1370]:

alpha_arr = [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2, 10**3, 10**4, 10**5]
alpha_dict = {'alpha': alpha_arr}

In [1371]:
gcv = GridSearchCV(Ridge(random_state=RANDOM_STATE), alpha_dict, refit=False)
gcv.fit(learn_df, learn_y)
gcv.best_params_


{'alpha': 100000}

In [1372]:
rcv = RidgeCV(alphas=alpha_arr)
rcv.fit(learn_df, learn_y)
predicted_rcv_y = rcv.predict(test_df)
rcv.alpha_


100000.0

In [1373]:
lcv = LassoCV(alphas = alpha_arr, random_state=RANDOM_STATE, selection='random')
lcv.fit(learn_df, learn_y)
predicted_lcv_y = lcv.predict(test_df)
lcv.alpha_

10.0

In [1374]:
glcv = GridSearchCV(Lasso(random_state=RANDOM_STATE, selection='random'), alpha_dict, refit=False)
glcv.fit(learn_df, learn_y)
glcv.best_params_

{'alpha': 1000}

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [1375]:
lasso_pipeline = make_pipeline(StandardScaler(), RidgeCV())
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

-2.2686242620594355

In [1376]:
lasso_pipeline = make_pipeline(MinMaxScaler(), RidgeCV())
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

-2.309348422060499

In [1377]:
lasso_pipeline = make_pipeline(StandardScaler(), LassoCV())
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

-2.4196633924469664

In [1378]:
lasso_pipeline = make_pipeline(MinMaxScaler(), LassoCV())
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

-2.4207616913002514

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [1379]:
lasso_pipeline = make_pipeline(StandardScaler(), RidgeCV(alphas=[ 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]))
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

-2.4266819554961137

In [1380]:
lasso_pipeline = make_pipeline(MinMaxScaler(), RidgeCV(alphas=[ 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]))
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

-2.406955450974433

In [1381]:
lasso_pipeline = make_pipeline(StandardScaler(), LassoCV(alphas=[ 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]))
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

-2.429411562364625

In [1382]:
lasso_pipeline = make_pipeline(MinMaxScaler(), LassoCV(alphas=[ 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05]))
lasso_pipeline.fit(learn_df, learn_y)
lasso_pipeline_predicted = lasso_pipeline.predict(test_df)
r2_score(test_y, lasso_pipeline_predicted)

-2.423438322246543

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [1383]:
pp = Pipeline(steps=[('scaler', StandardScaler()),('transform', PolynomialFeatures()), ('RidgeCV', RidgeCV())])
pp.fit(learn_df, learn_y)
predicted = pp.predict(test_df)
r2_score(test_y, predicted)


-2.4699656935499763

In [1384]:
pp = Pipeline(steps=[('scaler', MinMaxScaler()),('transform', PolynomialFeatures()), ('RidgeCV', RidgeCV())])
pp.fit(learn_df, learn_y)
predicted = pp.predict(test_df)
r2_score(test_y, predicted)


-2.2477921581491196

In [1385]:
pp = Pipeline(steps=[('scaler', StandardScaler()),('transform', PolynomialFeatures()), ('LassoCV', LassoCV())])
pp.fit(learn_df, learn_y)
predicted = pp.predict(test_df)
r2_score(test_y, predicted)

/Users/17299765/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.575143383903196, tolerance: 3.1607934413580248
  model = cd_fast.enet_coordinate_descent_gram(
/Users/17299765/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.23448858237316, tolerance: 3.1607934413580248
  model = cd_fast.enet_coordinate_descent_gram(
/Users/17299765/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.058399545570865, tolerance: 3.1607934413580248
  model = cd_fast.enet_coordinat

-2.429411562364625

In [1386]:
pp = Pipeline(steps=[('scaler', MinMaxScaler()),('transform', PolynomialFeatures()), ('LassoCV', LassoCV())])
pp.fit(learn_df, learn_y)
predicted = pp.predict(test_df)
r2_score(test_y, predicted)

/Users/17299765/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15.60232512096627, tolerance: 3.1607934413580248
  model = cd_fast.enet_coordinate_descent_gram(
/Users/17299765/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 27.510849290953047, tolerance: 3.1607934413580248
  model = cd_fast.enet_coordinate_descent_gram(
/Users/17299765/git/github/ds-practise/ds-practise/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 19.684963618812617, tolerance: 3.1607934413580248
  model = cd_fast.enet_coordi

-2.4388051296622266

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

http://archive.ics.uci.edu/ml/datasets/Adult

In [1387]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [1388]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

10. Выберите колонки с числовыми и категориальными переменными.

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.